# Get Elevation gradient data from Copernicus Digital Elevation Model (DEM)
Elevation data was provided for the ground measures but not for the test and train datasets. This notebook pulls the elevation gradient for the test and train grid cells and saves it into the data/static directory.

In [ ]:
!pip install pystac_client
!pip install planetary_computer
!pip install rasterio
!pip install xarray-spatial

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import pandas as pd
import numpy as np
import planetary_computer
import xarray
import time
from collections import defaultdict
from pystac_client import Client
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp
from matplotlib import pyplot as plt

### Import Base Data Files

In [ ]:
ground_measures_metadata = pd.read_csv('/content/drive/MyDrive/snocast/eval/data/ground_measures_metadata.csv')
submission_format = pd.read_csv('/content/drive/MyDrive/snocast/eval/data/submission_format.csv')

In [ ]:
# get latitude longitude for grids
f = open('/content/drive/MyDrive/snocast/eval/data/grid_cells.geojson')
grid_cells = json.load(f)
print('length grid_cells features: ', len(grid_cells['features']))

grid_features = defaultdict(dict)
for grid_cell in grid_cells['features']:
  cell_id = grid_cell['properties']['cell_id']
  coordinates = grid_cell['geometry']['coordinates'][0]
  region = grid_cell['properties']['region']
  grid_features[cell_id] = {'coordinates': coordinates[1:],
                            'region': region,
                            'geometry': grid_cell['geometry']}

ids = []
lats = []
lons = []
bboxes = []

for grid_cell in grid_cells['features']:
    cell_id = grid_cell['properties']['cell_id']
    coordinates = grid_cell['geometry']['coordinates'][0]
    lon, lat = np.mean(coordinates, axis=0)
    northeast_corner = np.max(coordinates, axis=0)
    southwest_corner = np.min(coordinates, axis=0)
    # bbox = [min_lon, min_lat, max_lon, max_lat]
    bbox = np.concatenate([southwest_corner,northeast_corner])
    ids.append(cell_id)
    lats.append(lat)
    lons.append(lon)
    bboxes.append(bbox)

grid_cells_pd = pd.DataFrame({'location_id': ids, 
                             'latitude': lats, 
                             'longitude': lons, 
                             'bbox': bboxes})

## Get Data for Copernicus Digital Elevation Model (DEM)

In [ ]:
def read_band(href, aoi):
    with rasterio.open(href) as ds:
        aoi_bounds = features.bounds(aoi)
        warped_aoi_bounds = warp.transform_bounds("epsg:4326", ds.crs, *aoi_bounds)
        aoi_window = windows.from_bounds(transform=ds.transform, *warped_aoi_bounds)
        try:
          data = ds.read(1, window=aoi_window)
        except:
          data = x = np.array([[0, 0],[0, 0]])
        return data

In [ ]:
client = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    ignore_conformance=True,
)

In [ ]:
df = grid_cells_pd

In [ ]:
df.head()

In [ ]:
aoi = grid_features['0001daba-dd41-4787-84ab-f7956f7829a8']['geometry']
aoi

In [ ]:
# Get all relevant items within the lat/lon bounds of the df
search = client.search(
    collections=["cop-dem-glo-30"],
    intersects=aoi,
)

items = list(search.get_items())
print(f"Returned {len(items)} items")

In [ ]:
signed_asset = planetary_computer.sign(items[0].assets["data"])
elev_matrix = read_band(signed_asset.href, aoi)

In [ ]:
plt.imshow(elev_matrix)
plt.colorbar()

In [ ]:
southern_gradient = -1*np.diff(elev_matrix, axis=0)
plt.imshow(southern_gradient)
plt.colorbar()
print(southern_gradient.mean())

In [ ]:
eastern_gradient = np.diff(elev_matrix, axis=1)
plt.imshow(eastern_gradient)
plt.colorbar()
print(eastern_gradient.mean())

In [ ]:
def get_elevations(df):
  east_grads = []
  south_grads = []
  east_pcts = []
  south_pcts = []
  ids = []

  for idx, row in df.iterrows():
    if idx % 250 == 0:
      print(idx)
    cell_id = row['location_id']
    aoi = grid_features[cell_id]['geometry']

    # Get all relevant items within the lat/lon bounds of the df
    search = client.search(
        collections=["cop-dem-glo-30"],
        intersects=aoi,
    )

    need_item = True
    num_tries = 0
    while need_item:
      try:
        items = list(search.get_items())
        need_item = False
      except:
        num_tries += 1
        print('exception')
        time.sleep(1)
        if num_tries > 3:
          need_item = False
          print('give up')
    
    if num_tries <= 3:
      loc_east_grads = []
      loc_south_grads = []
      loc_east_low = []
      loc_south_low = []
      loc_east_size = []
      loc_south_size = []
      for item in items:
        signed_asset = planetary_computer.sign(item.assets["data"])
        elev_matrix = read_band(signed_asset.href, aoi)
        eastern_grad = (np.diff(elev_matrix, axis=1))
        e_h, e_w = eastern_grad.shape
        east_low = (eastern_grad > 0).sum()
        east_size = e_h*e_w
        southern_grad = (-1*np.diff(elev_matrix, axis=0))
        s_h, s_w = southern_grad.shape
        south_low = (southern_grad > 0).sum()
        south_size = s_h*s_w
        loc_east_grads.append(eastern_grad.mean())
        loc_south_grads.append(southern_grad.mean())
        loc_east_low.append(east_low)
        loc_east_size.append(east_size)
        loc_south_low.append(south_low)
        loc_south_size.append(south_size)

      east_pct = np.sum(loc_east_low)/np.sum(loc_east_size)
      east_pcts.append(east_pct)
      east_grads.append(np.mean(loc_east_grads))
      south_pct = np.sum(loc_south_low)/np.sum(loc_south_size)
      south_pcts.append(south_pct)
      south_grads.append(np.mean(loc_south_grads))
      ids.append(cell_id)

  return east_grads, south_grads, ids, east_pcts, south_pcts


In [ ]:
test_east_grads, test_south_grads, test_ids, test_east_pcts, test_south_pcts = get_elevations(grid_cells_pd)

In [ ]:
test_elev_grads = pd.DataFrame({'cell_id': test_ids, 
                                 'east_elev_grad': test_east_grads, 
                                 'south_elev_grad': test_south_grads,
                                 'east_elev_pct': test_east_pcts,
                                 'south_elev_pct': test_south_pcts})

In [ ]:
test_elev_grads.to_parquet('/content/drive/MyDrive/snocast/eval/data/static/test_elevation_grads.parquet')